# Completion (02nd October 2023)

- Iterate by date (Some surveys have the same date)

# Clarifications

- Will the HEIFA scores be 1 decimal place? What if it fails to fall within the range?


# Questions to Ask

- Is it okay to use CSV file from your end as well? Ease of convenience and I don't have to hardcode it in my end

In [ ]:
from utils import *

import nest_asyncio
import asyncio

# Only run nest_asyncio in a Jupyter Notebook environment
nest_asyncio.apply()

In [ ]:
# Load the respective files

async def get_all_dataframes():
    return await asyncio.gather(
        load_intake24(),
        load_heifa_ingredients(),
        load_heifa_recipes(),
        load_heifa_scores()
    )

intake24_df, heifa_food_df, heifa_recipes_df, heifa_scores_df = asyncio.run(get_all_dataframes())

# Breakdown of Intake 24:

The file has many users.

Each user has many surveys.

Each survey has many meal intake.

Each intake consists of many food components.

Every food component is marked with a "Nutrition ID code".

In [ ]:
user_dict = create_user_objects(intake24_df)

for user_id in user_dict.keys():

    print(f"Printing for User {user_id}")
    user_obj = user_dict[user_id]
    user_obj.print_information()

# Breakdown of HEIFA (Food Composition)

Every row in the file is a unique ingredient.

Every ingredient:
- has it's own attributes.
- can be mapped to a 8-digit code (for HEIFA Recipe)
- is used as a divisor for either energy (kilo joules) or grams (g)

In [ ]:
# Create the objects
food_composition_dict = create_food_objects(heifa_food_df)

for key, food_comp_obj in food_composition_dict.items():

    food_comp_obj.print_full_details()

# Breakdown of HEIFA (Recipes)

- Every recipe has multiple ingredients
- Keys are repeated across rows (similar to Survey ID of Intake24)
- Every ingredient has respective proportion to the recipe

In [ ]:
recipe_dict = create_recipe_objects(heifa_recipes_df)

for id, recipe_obj in recipe_dict.items():
    print(f"Printing for ID {id}\n")
    recipe_obj.print_ingredients_information()

## Mapping between Intake24 and HEIFA Ingredients

- For each user, extract the given nutrients and store in an array.
- This is from ALL the survey data.
- We don't care about the order here.
- The array will contain a list of dictionaries/JSON.

In the array:

- Use the HEIFA ID (from user) to map to the HEIFA Ingredients' HEIFA ID.
- Check if a result is found or not.
- Check if it requires a recipe or not.

## Mapping between Intake24 and HEIFA Recipes

This is in case a recipe is found (The second step).

- For the given recipe, extract the given nutrients ID and proportion, store in an array.
- We don't care about the order here.
- The array will contain a list of dictionaries.

In the array:

- Use the HEIFA ID (from the recipes) to map the HEIFA Ingredients' HEIFA ID.
- Check the energy and serving size.

In [ ]:
user_daily_intake = calculate_user_servings(user_dict, food_composition_dict, recipe_dict)

In [ ]:
# Display
for user_id, daily_intake_dict in user_daily_intake.items():

    for date, food_group_dict in daily_intake_dict.items():

        print(f"Breakdown of User {user_id} on {date}:")

        for food_group, total_serving in food_group_dict.items():
            print(f"- {food_group}: {total_serving:.2f} serves")
        
        print("")

# Test with Samara's CSV file and post the updates here



# Calculating the HEIFA Scores

Heifa scores are to be calculated on a **daily basis**.

To calculate them, let's break them down:

- Break down by user
- Break down by date
- Break down by major food group (Example: Vegetables/Green -> Vegetables is the major food group)
- Break down by sub-food group of the major (Example: Vegetables/Green -> Green is the sub-food group)
- Compare the scores by gender (male and female)

There are some exceptions to the rule, based on the HEIFA scores guideline:

- Grains and cereals/Wholegrains -> This is to be calculated separately as "Grains and cereals" and "Wholegrains".

In [ ]:
# Create the HEIFA scores list

heifa_scores_dict = {}

def create_scores_object(scores_row):

    # Get the main attributes
    scores_dictionary = {
        'minimum_serves_male': scores_row['minimum_serves_male'],
        'maximum_serves_male': scores_row['maximum_serves_male'],
        'minimum_serves_female': scores_row['minimum_serves_female'],
        'maximum_serves_female': scores_row['maximum_serves_female'],
        'heifa_score': scores_row['heifa_score'],
    }

    food_group_list = heifa_scores_dict.get(scores_row['food_group'], [])
    food_group_list.append(scores_dictionary)
    heifa_scores_dict[scores_row['food_group']] = food_group_list

heifa_scores_df.apply(create_scores_object, axis=1)
print(heifa_scores_dict)


In [ ]:
from pprint import pprint

# Transform from individual serving to total servings
user_total_servings_dict = {}

for user_id, daily_intake_dict in user_daily_intake.items():

    date_servings_dict = {}

    for date, food_group_dict in daily_intake_dict.items():

        total_servings_dict = {}

        for food_group, total_serving in food_group_dict.items():
            
            print(f"Food group (Whole): {food_group}")
            sub_group = ""

            if "/" in food_group:
                food_group, sub_group = food_group.split("/")

            print(f"Food group (Main): {food_group}")
            print(f"Food group (Sub): {sub_group}")

            # Append to the dictionary
            total_servings = total_servings_dict.get(food_group, 0)
            total_servings += total_serving
            total_servings_dict[food_group] = total_servings
            print("\n")

        # Add to the date information
        date_servings_dict[date] = total_servings_dict
    
    user_total_servings_dict[user_id] = date_servings_dict
        
        


pprint(user_total_servings_dict)


In [ ]:
# Transform from serving size to HEIFA score

def range_found(range_list, serving_size):
    minimum, maximum = range_list
    return serving_size >= minimum and serving_size <= maximum

def find_heifa_scores(food_group, serving_size):

    male_score, female_score = 0,0

    male_serving_keys = ['minimum_serves_male', 'maximum_serves_male']
    female_serving_keys = ['minimum_serves_female', 'maximum_serves_female']

    # Extract the list to see
    scores_list = heifa_scores_dict[food_group]

    for score_dict in scores_list:

        # Find the scores (male)
        range_found_male = range_found([score_dict[key] for key in male_serving_keys], serving_size)
        male_score = score_dict['heifa_score'] if (range_found_male and score_dict['heifa_score'] > male_score) else male_score

        # Find the scores (female)
        range_found_female = range_found([score_dict[key] for key in female_serving_keys], serving_size)
        female_score = score_dict['heifa_score'] if (range_found_female and score_dict['heifa_score'] > female_score) else female_score

    
    return male_score, female_score

for user_id, daily_intake_dict in user_daily_intake.items():

    for date, food_group_dict in daily_intake_dict.items():

        print(f"Breakdown of User {user_id} on {date}:")

        for food_group, total_serving in food_group_dict.items():
            print(f"- {food_group}: {total_serving:.2f} serves")

            if food_group in heifa_scores_dict:

                male_heifa_score, female_heifa_score = find_heifa_scores(food_group, round(total_serving, 1))

                print(f"- HEIFA Score (Male): {male_heifa_score}")
                print(f"- HEIFA Score (Female): {female_heifa_score}")

            print("\n")